# ATOC 5860 Homework 1
## Genevieve Clow

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

# 1) Basic Statistics

## 1. a)

*Bayes Theorem. Assume background rates of COVID are 90% negative, 10% positive
AND COVID tests are accurate 80% of the time, but fail 20% of the time. Your friend goes
and gets a COVID test. Your friend test negative. What is the probability that your friend
is actually negative? Explain to your friend how you are using Bayes theorem to inform
your thinking. Hint: Review Lecture #1 and the 1.2.2.2 of the Barnes Notes. (10 points)*

**Barnes eq. 23**

$$
Pr(N|T) = \frac{Pr(T|N)Pr(N)}{Pr(T|N)Pr(N)+Pr(T|P)Pr(P)}
$$

* Pr(N|T) = probability of being negative given a negative test result
* Pr(T|N) = probability of an accurate test = 80%
* Pr(T|P) = probability of an inaccurate test = 20%
* Pr(N) = background rate of negative tests = 90%
* Pr(P) = background rate of positive tests = 10%

In [2]:
pr_n_t = (0.8*0.9)/((0.8*0.9)+(0.2*0.1))
print(pr_n_t)

0.9729729729729729


There is a 97% chance that your friend is actually negative, given their negative result. 

Using Bayes theorem, we can use our knowledge of background COVID rates to improve our interpretation of the test result. Without knowing the background rates, we would be 80% confident in the test result. However, given that the background rate is so low, we can be even more confident that it is a true negative. 

## 1. b)

*Explain how to test whether a sample mean is significantly different than zero at the
95% confidence level and the 99% confidence level. State each of the 5 steps in
hypothesis testing that you are using. For step 4, calculate the specific critical value
assuming a two-tailed test. Contrast your approach for a sample with 15 independent
observations (N=15) and a sample 1000 independent observations (N=1000). (15 points)*

We can determine how much the sample mean deviates from the population mean by calculating the z (large N) or t (small N) statistic. These values indicate the number of standard errors that our sample means lies from the population mean. To determine if the difference is statistically significant, we follow the steps below.

Steps in Hypothesis Testing: 
1. State the signicance level: 
    - For the 95% confidence level: alpha = 0.05 
    - For the 99% confidence level: alpha = 0.01
2. State the null hypothesis (H0) and the alternative (H1)
    - H0 = The (standardized) sample mean is equal to zero
    - H1 = The sample mean is not equal to zero
3. State the statistic to be used, and the assumptions required to use it
    - For N = 15: use the t-statistic becuase we have a small sample size (<30)
    - For N = 1000: use the z-statistic becuase we have a large sample size 
    - Assumptions: the data variables are normally distributed and the samples are independent. 
4. State the critical region
    - For the 95% confidence level: tcrit = 2.14, zcrit = 1.96
    - For the 99% confidence level: tcrit = 2.97, zcrit = 2.57
5. Evaluate the statistic and state the conclusion
    - Calculate the t and z statistics using the equations below
    - If abs(t) > tcrit or abs(z) > zcrit, then we can reject the null hypothesis. This means that there is 5% (or 1%) probability of getting the sample mean by chance given the population. 

    \begin{align}
    z = \frac{\bar{X} - \mu}{\frac{\sigma}{\sqrt{N}}}
    \end{align}

    - Where $\bar{X}$ = sample mean, $\sigma$ = population standard deviation, $\mu$ = population mean, N = sample size

    $$
    t = \frac{\bar{x} - \mu}{\frac{s}{\sqrt{N-1}}}
    $$

    - Where $\bar{x}$ = sample mean, s = sample standard deviation, $\mu$ = population mean, N = sample size

In [3]:
print('95% confidence level')
alpha = 0.05 
# python calculates lower tail probability, we want two-sided test
p = (1-alpha/2)

# t-stat
# df = 15 -1
tcrit = stats.t.ppf(p, 14) 
print('t-stat critical value (N = 15):', round(tcrit,4))

# z-stat
zcrit = stats.norm.ppf(p)
print('z-stat critical value (N = 1000):', round(zcrit,4))

95% confidence level
t-stat critical value (N = 15): 2.1448
z-stat critical value (N = 1000): 1.96


In [4]:
print('99% confidence level')
alpha = 0.01 
# python calculates lower tail probability, we want two-sided test
p = (1-alpha/2)

# t-stat
# df = 15 -1
tcrit = stats.t.ppf(p, 14)
print('t-stat critical value (N = 15):', round(tcrit,4))

# z-stat
zcrit = stats.norm.ppf(p)
print('z-stat critical value (N = 1000):', round(zcrit,4))

99% confidence level
t-stat critical value (N = 15): 2.9768
z-stat critical value (N = 1000): 2.5758


## 1. c)  

*Design your own homework problem to compare two sample means using data of your
own choice. In other words, test whether two sample means are statistically different.
Follow all five steps of hypothesis testing. Hint: See page 26 of Barnes notes for an
example. (15 points)*

### Compare the global mean of surface chlorophyll derived from two satellie emulators (using MODIS and ISCCP clouds)

Since chlorophyll is log-normally distributed, I attempted to apply the central limit theorem. Instead of using the raw chlorophyll values (daily), I first took the annual global average. Then, I compared the means of the annual data. Unfortunately, we only have 30 years of data to work with, so this is a small sample size and the central limit theorem may not apply.

Hypothesis testing: 
1. State the signicance level: 95% (alpha = 0.05)
2. State the null hypothesis H0 and the alternative H1
- H0 = The global mean of the two chlorophyll samples are equal.
- H1 = The global mean of the two chlorophyll samples are NOT equal.
3. State the statistic to be used, and the assumptions required to use it
- I will use a two-sided t-test. Since I am using annual means, the small sample sizes are small (30). I am assuming that the two samples are drawn from normal distributions with equal standard deviations. 
4. State the critical region
- tcrit = 2.045 (if t > tcrit, reject the null hypothesis)
5. Evaluate the statistic and state the conclusion

In [5]:
# Determine the critical value
# We have 30 years of data for each sample
# N1 = N2 = 30
N = 30 
df = N - 1 
tcrit = stats.t.ppf(0.975,df)
print(tcrit)

2.045229642132703


### Import data

In [6]:
TAREA = xr.open_dataset('/glade/scratch/gclow/archive/b.e22.B1850.f09_g17.cosp_chlor_30yr/ocn/hist/b.e22.B1850.f09_g17.cosp_chlor_30yr.pop.h.nday1.0001-01-01.nc').TAREA

base = '/glade/scratch/gclow/archive/b.e22.B1850.f09_g17.cosp_chlor_30yr/ocn/daily/b.e22.B1850.f09_g17.cosp_chlor_30yr.pop.h.ecosys.nday1.'

data = xr.open_mfdataset(base+'*.nc', concat_dim="time", parallel = True, chunks = {'time':120},
                              data_vars='minimal', compat='override', 
                              coords='minimal')

FileNotFoundError: [Errno 2] No such file or directory: '/glade/scratch/gclow/archive/b.e22.B1850.f09_g17.cosp_chlor_30yr/ocn/hist/b.e22.B1850.f09_g17.cosp_chlor_30yr.pop.h.nday1.0001-01-01.nc'

### Calculate weighted global mean for each year

In [ ]:
isccp_global_mean = ((data.totChl_isccp*TAREA).groupby('time.year').sum(dim = ['time','nlat','nlon'])/(data.totChl_isccp_wgt*TAREA).groupby('time.year').sum(dim = ['time','nlat','nlon'])).compute()
modis_global_mean = ((data.totChl_modis*TAREA).groupby('time.year').sum(dim = ['time','nlat','nlon'])/(data.totChl_modis_wgt*TAREA).groupby('time.year').sum(dim = ['time','nlat','nlon'])).compute()

In [ ]:
isccp_mean = isccp_global_mean.mean().data
isccp_std = isccp_global_mean.std().data
isccp_var = isccp_global_mean.var().data

print('ISCCP Mean:',isccp_mean)
print('ISCCP Standard Dev.:',isccp_std)
xr.plot.hist(isccp_global_mean);

In [ ]:
modis_mean = modis_global_mean.mean().data
modis_std = modis_global_mean.std().data
modis_var = modis_global_mean.var().data

print('MODIS Mean:',modis_mean)
print('MODIS Standard Dev.:',modis_std)
xr.plot.hist(modis_global_mean);

The means are not normally distributed. Therefore, we may be violating an important assumption in our test.

### Calculate the t statistic
**Barnes Eq. 101**

$t = \frac{\bar{x_{1}} - \bar{x_{2}}}{\hat{\sigma}\sqrt{\frac{1}{N_{1}} + \frac{1}{N_{2}}}}$

where
$\hat{\sigma} = \sqrt{\frac{N_{1}s_{1}^2+N_{2}s_{2}^2}{N_{1}+N_{2}-2}}$

In [ ]:
sigma = np.sqrt((N*isccp_var + N*modis_var)/(N+N-2))
print(sigma)

In [ ]:
t = (isccp_mean - modis_mean)/(sigma*np.sqrt((1/N)+(1/N)))
print(t)

Conclusion: Since t > tcrit, we can reject the null hypothesis. This means that there is a 95% chance that the difference in means did not occur by chance. However, I am not super confident in this result becuase the sample size is small and the variables are not normally distributed.

## 1. d)

*Design your own homework problem to place 95% confidence intervals on the mean
value of a data variable of your choice. Use the non-standardized variable. Hint: See
Barnes notes on Confidence Intervals.*

I calculated the confidence intervals on the simulated chlorophyll observations (derived from ISCCP clouds) using the equation below.

**Barnes Eq. 100** 

$$
\mu = \bar{x} \pm t_{c}\frac{s}{\sqrt{N-1}}
$$

In [ ]:
# Calculate t_crit
N = 30 
df = N - 1 
tcrit = stats.t.ppf(0.975,df)
print(tcrit)

In [ ]:
# Calculate upper and lower limits
diff = tcrit*(isccp_std/np.sqrt(N-1))

In [ ]:
print(round(isccp_mean-diff,3), '<= \u03BC <=', round(isccp_mean+diff,3))

## 1. e)

*The F-statistic is used to compare two sample standard deviations. Design your own
homework problem to compare two sample standard deviations and assess if they are
different at the 95% confidence interval. Hint: See page 38 of the Barnes notes. Note:
When calculating the f-statistic Barnes Chapter 1 Equation (122), the larger variance
should always be on top (numerator) and the smaller variance should always be on
bottom (denominator). i.e., F = Larger variance / Smaller variance. (10 points)*

The F-statistic is used to compare two sample standard deviations. Design your own
homework problem to compare two sample standard deviations and assess if they are
different at the 95% confidence interval. Hint: See page 38 of the Barnes notes. Note:
When calculating the f-statistic Barnes Chapter 1 Equation (122), the larger variance
should always be on top (numerator) and the smaller variance should always be on
bottom (denominator). i.e., F = Larger variance / Smaller variance.

Steps:
1) State significance level. alpha=0.05; 95% confidence
2) Null Hypothesis = The standard deviation for both chlorophyll samples are the same.
3) We will use the f-statistic. We will assume data1 and data2 come from normal populations having the same true variance.
4) Find critical value to exceed to reject the null hypothsis (fcrit)

**Barnes Eq. 122** 

$$
F = \frac{s_{1}^2}{s_{2}^2}
$$

In [ ]:
# use 0.975 because this function calculates the lower-tail probability
fcrit = stats.f.ppf(q = 0.975, dfn = N-1, dfd = N-1)
print(fcrit)

In [ ]:
f = isccp_var/modis_var
print(f)

In [ ]:
abs(f) > fcrit

We cannot reject our null hypothesis that the two samples have different variances

# 2) Bootstrapping

*Compare composite-averages using t/z tests and bootstrapping. Note: coding is required
for this problem. Please use python Jupyter notebooks. It will be helpful follow the ipython
notebook examples introduced in Application Lab #1 and in lectures. (40 points)
Your friend living in Fort Collins tells you that the air pressure is anomalous when there is
measurable precipitation (greater than or equal to 0.01 inches). To test your friends’
hypothesis, use hourly observations from Fort Collins in 2014. The data include both the
precipitation amount in units of inches and pressure in units of hPa at hourly frequency. The
data file is called homework1_data.csv.*

### Load data and make exploratory plots

In [ ]:
data = pd.read_csv('homework1_data.csv')
data.head()

In [ ]:
data['P_hPa'].plot()
plt.xlabel('Time (hr)');
plt.ylabel('Pressure (hPA)');

In [ ]:
plt.hist(data['P_hPa'])
plt.xlabel('Pressure (hPa)');
plt.ylabel('Count');

In [ ]:
data['R_inches'].plot()
plt.xlabel('Time (hr)');
plt.ylabel('Rain (in)');

In [ ]:
plt.hist(data['R_inches'])
plt.xlabel('Rain (in)');
plt.ylabel('Count');

## 2.a) 
*What was the average pressure in 2014? What was the average pressure when it
rained? (10 points)*

In [ ]:
# Average pressure 
data['P_hPa'].mean()

In [ ]:
# Average pressure when it rained
data_rain = data[data['R_inches']>=0.01].copy()
data_rain['P_hPa'].mean()

The average pressure in 2014 was 846.33 hPa. When it rained, the average pressure was 847.03 hPa.

## 2.b)

*Test your friends’ hypothesis by generating confidence intervals using both a t-statistic
and a z-statistic. Is the average pressure different when it is raining? What is more
appropriate to use as a statistical test – a t- or a z-statistic? Use 95% confidence interval.
(15 points)*

**For the t-test:** 

$$
\mu = \bar{x} \pm t_{c}\frac{s}{\sqrt{N-1}}
$$

**For the z-test:** 

$$
\mu = \bar{x} \pm z_{c}\frac{s}{\sqrt{N}}
$$

In [ ]:
N = data_rain['P_hPa'].count()
print(N)

In [ ]:
tcrit = stats.t.ppf(0.975, N-1)
zcrit = stats.norm.ppf(0.975)

In [ ]:
# z-stat
low_lim = data_rain['P_hPa'].mean()-zcrit*(data_rain['P_hPa'].std()/np.sqrt(N))
high_lim = data_rain['P_hPa'].mean()+zcrit*(data_rain['P_hPa'].std()/np.sqrt(N))
print('z-stat confidence interval: ', round(low_lim,3), '-', round(high_lim,3))

In [ ]:
# t-stat
low_lim = data_rain['P_hPa'].mean()-tcrit*(data_rain['P_hPa'].std()/np.sqrt(N-1))
high_lim = data_rain['P_hPa'].mean()+tcrit*(data_rain['P_hPa'].std()/np.sqrt(N-1))
print('t-stat confidence interval: ', round(low_lim,3), '-', round(high_lim,3))

The mean of all data (846.332) lies outside of the 95% confidence range of the rainy data. The z-statistic and t-statistic are very similar in this case, so I would argue that either would be appropriate. This indicates that we have a large enough sample where sample size doesn't significantly change our results.

However, I suspect that there is high auto-correlation in this dataset, so the true number of independent samples may be lower. 

## 2.c)

*Instead of the t/z-test – use bootstrap sampling to determine whether the local
pressure is anomalously high during times when it is raining. How does your answer
compare with your results using the t/z-test? (15 points)*

In [ ]:
N_b = 1000

## initialize array
P_Bootstrap=np.empty((N_b,N))

## loop over to fill in array with randomly selected values
for i in range(N_b):
    P_Bootstrap[i,:]=np.random.choice(data['P_hPa'],N)

In [ ]:
## Calculate the means of your randomly selected SWE values.
P_Bootstrap_mean=np.mean(P_Bootstrap,axis=1)

P_Bootstrap_mean_avg=np.mean(P_Bootstrap_mean)
print('Mean:',P_Bootstrap_mean_avg)
P_Bootstrap_mean_std=np.std(P_Bootstrap_mean)
print('Standard Dev:',P_Bootstrap_mean_std)
P_Bootstrap_mean_min=np.min(P_Bootstrap_mean)
print('Min:', P_Bootstrap_mean_min)
P_Bootstrap_mean_max=np.max(P_Bootstrap_mean)
print('Max:', P_Bootstrap_mean_max)

In [ ]:
plt.hist(P_Bootstrap_mean)
plt.xlabel('Mean Pressure (hPa)');
plt.ylabel('Count');
plt.title('Bootstrapped Randomly Selected Mean Pressure Values');

**Barnes eq. 83**

$$
z = \frac{\bar{\chi} - \mu}{\frac{\sigma}{\sqrt{N}}}
$$

where 
- $\bar{\chi}$ = sample mean 
- $\mu$ = population mean 

In [ ]:
# sample_N is 1 because we are comparing the sample mean to a distribution of means
sample_N = 1
sample_mean=data_rain['P_hPa'].mean() # rainy days only
population_mean=np.mean(P_Bootstrap_mean)
population_std=np.std(P_Bootstrap_mean)
xstd=population_std/np.sqrt(sample_N)
z =(sample_mean-population_mean)/xstd
print(z)

In [ ]:
zcrit = stats.norm.ppf(0.975)
print(zcrit)

z > zcrit, so we can reject the null hypothesis. This is consistent with what we found previously. 